# 🧠 Roadmap — Sistema de Perguntas e Respostas com Transformers

---

# 🤗 Pipeline de QA com modelo treinado em português

In [ ]:
from transformers import pipeline

# Modelo BERT em português treinado em QA
qa_pipeline = pipeline(
    task="question-answering",
    model="mrm8488/bert-base-portuguese-cased-finetuned-squad-v1",
    tokenizer="mrm8488/bert-base-portuguese-cased-finetuned-squad-v1"
)

contexto = """
A Amazônia é a maior floresta tropical do mundo. Ela cobre nove países e representa uma das maiores reservas de biodiversidade do planeta.
"""

pergunta = "Qual é a maior floresta tropical do mundo?"

resposta = qa_pipeline(question=pergunta, context=contexto)
print("Resposta:", resposta["answer"])


# ✅ Parte 2 — Fine-Tuning com modelo extractivo (SQuAD em português)

In [ ]:
from datasets import load_dataset

dataset = load_dataset("squad", split="train[:1000]")
dataset = dataset.train_test_split(test_size=0.1)
dataset["validation"] = dataset["test"]


# 🔤 Tokenização e alinhamento de contexto/pergunta

In [ ]:
from transformers import AutoTokenizer

checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def preprocess(examples):
    return tokenizer(
        examples["question"],
        examples["context"],
        truncation=True,
        padding="max_length",
        max_length=384
    )

tokenized_datasets = dataset.map(preprocess, batched=True)


# 🧠 Carregar modelo para QA extractivo

In [ ]:
from transformers import AutoModelForQuestionAnswering

model = AutoModelForQuestionAnswering.from_pretrained(checkpoint)


# ⚙️ Configurar treinamento

In [ ]:
from transformers import TrainingArguments

args = TrainingArguments(
    output_dir="./qa-model",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=2,
    weight_decay=0.01
)


# 🧪 Avaliação com F1 e EM

In [ ]:
import evaluate

metric = evaluate.load("squad")

def compute_metrics(p):
    return metric.compute(predictions=p.predictions, references=p.label_ids)


# 🏋️ Treinamento com Trainer

In [ ]:
from transformers import Trainer, DefaultDataCollator

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=DefaultDataCollator(),
    compute_metrics=compute_metrics
)

trainer.train()


# ✅ Avaliação final

In [ ]:
trainer.evaluate()


# 📌 Observações
* A tarefa de QA extractivo assume que a resposta está contida literalmente no contexto.

* Para QA generativo (ex.: com T5), use o prefixo "pergunte:" e modelo de geração seq2seq.

* Métricas como Exact Match (EM) e F1-score são as mais usadas para QA.